This notebook georeferences your image times. This is useful for visualising them in a GIS, but also for the making_training_data.ipynb, as that notebook assumes your tiles are georeferenced so that they can be matched to the location of your caravan polygons. This notebook assumes you have a relevant CSV which identifies each image with a name, its local path and holds its x and y extents.

In [20]:
import os 
import pandas as pd
from shutil import copy
import rasterio 
import warnings

In [5]:
#reads in the csv of all the image tiles that correspond with the sample of caravans from MAIA topo
df = pd.read_csv(".../caravanProject/imagery/km_tiles_maia_topo_caravans.csv", delimiter=',')
df.head()

,id,tile_name,geom,min_x,min_y,max_x,max_y,bbox
0,152500,SH7719,0106000020346C00000100000001030000000100000005...,276999.9971,318999.9965,277999.9993,320000.0002,"276999.997102446,318999.996456097,277999.99933..."
1,249036,TF0910,0106000020346C00000100000001030000000100000005...,508999.9963,310000.0027,509999.9986,310999.9964,"508999.996333681,310000.002677518,509999.99856..."
2,40524,NH6148,0106000020346C00000100000001030000000100000005...,261000.0026,848000.0018,261999.9965,848999.9955,"261000.002557401,848000.001779718,261999.99654..."
3,272055,TM3764,0106000020346C00000100000001030000000100000005...,637000.0022,264000.0000,637999.9962,265000.0038,"637000.002164408,264000.000041303,637999.99615..."
4,272402,TM4271,0106000020346C00000100000001030000000100000005...,641999.9969,270999.9963,642999.9991,272000.0001,"641999.99685252,270999.996313214,642999.999088..."


In [22]:
#this function returns the local filepath of the images
def filepath_local(tile_name):
    extension = '.jpg'
    path = ".../caravanProject/imagery/images_maia_topo_polygons/"
    return path + tile_name + extension 

In [23]:
#then the local filepath is added as a column to the dataframe of all images corresponding to maia topo caravan polygons
df['localpath'] = df['tile_name'].apply(filepath_local)
df.head()

,id,tile_name,geom,min_x,min_y,max_x,max_y,bbox,localpath,geotiff
0,152500,SH7719,0106000020346C00000100000001030000000100000005...,276999.9971,318999.9965,277999.9993,320000.0002,"276999.997102446,318999.996456097,277999.99933...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
1,249036,TF0910,0106000020346C00000100000001030000000100000005...,508999.9963,310000.0027,509999.9986,310999.9964,"508999.996333681,310000.002677518,509999.99856...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
2,40524,NH6148,0106000020346C00000100000001030000000100000005...,261000.0026,848000.0018,261999.9965,848999.9955,"261000.002557401,848000.001779718,261999.99654...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
3,272055,TM3764,0106000020346C00000100000001030000000100000005...,637000.0022,264000.0000,637999.9962,265000.0038,"637000.002164408,264000.000041303,637999.99615...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
4,272402,TM4271,0106000020346C00000100000001030000000100000005...,641999.9969,270999.9963,642999.9991,272000.0001,"641999.99685252,270999.996313214,642999.999088...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...


In [9]:
#create a list of all the tile_names of the images that correspond to maia topo caravan polygons
tile_names=[]
directory = ".../caravanProject/imagery/images_maia_topo_polygons"
for i in os.listdir(directory):
    if i.endswith(".jpg"):
        tile_names.append(i[:-4])
print(tile_names)

['NH6148', 'NN9613', 'NO2423', 'NS1984', 'SH4393', 'SH5866', 'SH5921', 'SH6112', 'SH7100', 'SH7219', 'SH7317', 'SH7318', 'SH7417', 'SH7418', 'SH7419', 'SH7501', 'SH7506', 'SH7518', 'SH7601', 'SH7701', 'SH7719', 'SH7801', 'SH7802', 'SH8053', 'SH8515', 'SH8613', 'SJ6565', 'SJ6765', 'SJ7163', 'SJ7263', 'SN5677', 'SN5884', 'SN5979', 'SN5984', 'SN6292', 'SN6376', 'SN7098', 'SN7476', 'SN7477', 'SN7478', 'SN7895', 'SN7995', 'SP8160', 'TF0909', 'TF0910', 'TF1008', 'TF1009', 'TF1010', 'TF1108', 'TF1119', 'TF1404', 'TF1414', 'TF1508', 'TF1602', 'TF1605', 'TF1611', 'TF1704', 'TF1705', 'TF1804', 'TF1805', 'TF1808', 'TF1902', 'TF2100', 'TF2101', 'TF2102', 'TF2103', 'TF2105', 'TF2106', 'TF2202', 'TF2210', 'TF2217', 'TF2302', 'TF2519', 'TF3015', 'TF3108', 'TF3115', 'TF3209', 'TF3214', 'TF3224', 'TF3227', 'TF3308', 'TF3309', 'TF3310', 'TF3311', 'TF3312', 'TF3404', 'TF3405', 'TF3408', 'TF3409', 'TF3412', 'TF3414', 'TF3419', 'TF3420', 'TF3424', 'TF3426', 'TF3427', 'TF3504', 'TF3507', 'TF3509', 'TF3607',

In [21]:
#georeference the images
warnings.simplefilter('ignore')
for idx, row in df.iterrows():
    if row['tile_name'] in tile_names:         
        trans=rasterio.Affine.from_gdal(row['min_x'], 0.25, 0, row['max_y'], 0, -0.25)
        crs={'init':'epsg:27700'}
        raster = rasterio.open(row['localpath'])
        im = raster.read()
        filestring = localdrive+"/caravanProject/imagery/images_maia_topo_polygons/geo_tiffs/" + row['tile_name'] + ".jpg"
        height= raster.height
        width= raster.width
        output_image= rasterio.open(filestring, 'w', driver='GTiff', compress='JPEG', height=height, width=width, count=3, dtype='uint8', crs=crs, transform=trans)
        output_image.write(im)
        raster.close()
        output_image.close()

In [13]:
#get the geotiff path
def geotiff_filepath(tile_name): # SH7719
    extension = '.jpg'
    geotiff_directory = ".../caravanProject/imagery/images_maia_topo_polygons/geo_tiffs/"
    return geotiff_directory + tile_name + extension 

#then the geotiff file is added as a column to the dataframe of all images corresponding to maia topo caravan polygons
df['geotiff'] = df['tile_name'].apply(geotiff_filepath)
df.head()

,id,tile_name,geom,min_x,min_y,max_x,max_y,bbox,localpath,geotiff
0,152500,SH7719,0106000020346C00000100000001030000000100000005...,276999.9971,318999.9965,277999.9993,320000.0002,"276999.997102446,318999.996456097,277999.99933...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
1,249036,TF0910,0106000020346C00000100000001030000000100000005...,508999.9963,310000.0027,509999.9986,310999.9964,"508999.996333681,310000.002677518,509999.99856...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
2,40524,NH6148,0106000020346C00000100000001030000000100000005...,261000.0026,848000.0018,261999.9965,848999.9955,"261000.002557401,848000.001779718,261999.99654...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
3,272055,TM3764,0106000020346C00000100000001030000000100000005...,637000.0022,264000.0000,637999.9962,265000.0038,"637000.002164408,264000.000041303,637999.99615...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
4,272402,TM4271,0106000020346C00000100000001030000000100000005...,641999.9969,270999.9963,642999.9991,272000.0001,"641999.99685252,270999.996313214,642999.999088...",.../caravanProject/imagery/images_maia_topo_po...,.../caravanProject/imagery/images_maia_topo_po...
